# Setup: Initialize neuprint client

User must input own neuprint token

In [ ]:
# Importing and calling neuprint api
from neuprint import Client
c = Client('neuprint.janelia.org', dataset='hemibrain:v1.2.1', token='[insert token here]') 
c.fetch_version()

# Get neuron criteria

In [15]:
from neuprint import NeuronCriteria as NC
from neuron_criteria import getClock

In [16]:
clock_df = getClock()

In [ ]:
from neuprint import fetch_neurons, NeuronCriteria as NC
criteria = NC(bodyId=tuple(clock_df['bodyId']))
LNdcriteria = NC(type='LNd.*', regex=True)
LNd_neurons = fetch_neurons(LNdcriteria)

# Fetch synapse information for LNd neurons and all pre and post connections

In [19]:
from neuprint import fetch_simple_connections

pre_conns = fetch_simple_connections(None,criteria)
post_conns = fetch_simple_connections(criteria)

In [21]:
from neuprint import fetch_synapses

LNd_synapses = fetch_synapses(LNdcriteria)
LNd_synapses

,bodyId,type,roi,x,y,z,confidence
0,5813064789,pre,SMP(R),20138,21479,7759,0.974000
1,5813064789,pre,SMP(R),20278,21266,7870,0.976000
2,5813064789,pre,SMP(R),20230,21276,7934,0.694000
3,5813064789,pre,SMP(R),20254,21310,7902,0.986000
4,5813064789,pre,SMP(R),20222,21247,7938,0.935000
...,...,...,...,...,...,...,...
9326,448260940,post,SMP(R),16605,21609,4816,0.730888
9327,448260940,post,SMP(R),16730,17825,8542,0.988277
9328,448260940,post,SLP(R),13860,21757,6023,0.827187
9329,448260940,post,SLP(R),13823,21795,5996,0.968956


# Sort information from highest to lowest number of synaptic connections per neuron

In [22]:
# The main thing here is to sort this information from highest to lowest number of synaptic connections for each neuron

post_conns_small = post_conns[['bodyId_pre','bodyId_post','instance_pre','instance_post','weight']]

post_conns_sort = post_conns_small.sort_values('weight', ascending = False)

pre_conns_small = pre_conns[['bodyId_pre','bodyId_post','instance_pre','instance_post','weight']]

pre_conns_sort = pre_conns_small.sort_values('weight', ascending = False)

In [21]:
import pandas as pd

def ranked_lists(clock_df, conns_sort, pre_or_post):
    """
    Generates a dataframe with each 'type' of pre or postsynaptic neurons ranked by weight
    
    :param clock_df: clock dataframe
    :param conns_sort: dataframe of connection information sorted from highest to lowest number of synaptic connections for each neuron
    :param pre_or_post: string determining if it's looking at synaptic inputs (presynaptic) or targets (postsynaptic)
    :return:
    """
    tables = []
    #looking at one particular type at a time
    types = clock_df['type'].unique()
    for group in types:
        #these are all the body Ids of that type
        IDs = clock_df[clock_df['type']==group]['bodyId']
        count = 1
        #looking at one neuron at a time
        for ID in IDs:
            #takes only the rows where the presynaptic neuron is that neuron
            if pre_or_post == 'post':
                sub_conns = conns_sort[conns_sort["bodyId_pre"] == ID]
            if pre_or_post == 'pre':
                sub_conns = conns_sort[conns_sort["bodyId_post"] == ID]
                sub_conns = sub_conns[['bodyId_pre', 'instance_pre', 'weight']]
            sub_conns.reset_index(drop=True, inplace=True)
            #adds on that information onto post_tables
            tables.append(sub_conns)
            count = count + 1

    all_grouped = pd.concat(tables, axis = 1)
    return all_grouped

In [22]:
all_post = ranked_lists(clock_df, post_conns_sort, 'post')
all_pre = ranked_lists(clock_df, pre_conns_sort, 'pre')

#saves the tables as csvs
all_post.to_csv('ranked_post_temp.csv')
all_pre.to_csv('ranked_pre_temp.csv')

In [ ]:
# Use merge_neuron_properties and connection_table_to_matrix to organize the data 